<h1>Deploy Gradio Apps to Hugging Face Spaces</h1>

This notebook will teach you how to create and deploy Gradio applications to Hugging Face Spaces in simple steps.

## What you'll learn:
- Create a simple Gradio app
- Test it locally
- Deploy to Hugging Face Spaces
- Best practices and tips

## 📦 Step 1: Install Required Libraries

First, let's install Gradio and other necessary packages.

In [ ]:
# Install required packages
!pip install gradio
!pip install huggingface_hub

## 🎯 Step 2: Create Your First Gradio App

Let's start with a simple text processing app that converts text to uppercase and counts words.

In [ ]:
import gradio as gr

def process_text(text):
    """
    Simple function that processes text:
    - Converts to uppercase
    - Counts words
    - Returns both results
    """
    if not text:
        return "Please enter some text!", 0

    uppercase_text = text.upper()
    word_count = len(text.split())

    return uppercase_text, word_count

# Test the function
test_result = process_text("Hello world! This is a test.")
print(f"Uppercase: {test_result[0]}")
print(f"Word count: {test_result[1]}")

Uppercase: HELLO WORLD! THIS IS A TEST.
Word count: 6


## 🎨 Step 3: Create the Gradio Interface

Now let's create a beautiful Gradio interface for our function.

In [ ]:
# Create the Gradio interface
demo = gr.Interface(
    fn=process_text,
    inputs=[
        gr.Textbox(
            label="Enter your text",
            placeholder="Type something here...",
            lines=3
        )
    ],
    outputs=[
        gr.Textbox(label="Uppercase Text", lines=3),
        gr.Number(label="Word Count")
    ],
    title="📝 Text Processor",
    description="Enter some text and I'll convert it to uppercase and count the words!",
    examples=[
        ["Hello world!"],
        ["Gradio makes machine learning demos easy!"],
        ["This is a sample text with multiple words to demonstrate the word counting feature."]
    ],
    theme=gr.themes.Soft()
)

# Launch the app locally
if __name__ == "__main__":
    demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a3712d5e96c3bf6e1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a3712d5e96c3bf6e1b.gradio.live


## 📁 Step 4: Prepare Files for Deployment

To deploy to Hugging Face Spaces, we need to create specific files. Let's create them:

In [ ]:
import os

# Create the main app file
app_code = '''import gradio as gr

def process_text(text):
    """
    Simple function that processes text:
    - Converts to uppercase
    - Counts words
    - Returns both results
    """
    if not text:
        return "Please enter some text!", 0

    uppercase_text = text.upper()
    word_count = len(text.split())

    return uppercase_text, word_count

# Create the Gradio interface
demo = gr.Interface(
    fn=process_text,
    inputs=[
        gr.Textbox(
            label="Enter your text",
            placeholder="Type something here...",
            lines=3
        )
    ],
    outputs=[
        gr.Textbox(label="Uppercase Text", lines=3),
        gr.Number(label="Word Count")
    ],
    title="📝 Text Processor",
    description="Enter some text and I'll convert it to uppercase and count the words!",
    examples=[
        ["Hello world!"],
        ["Gradio makes machine learning demos easy!"],
        ["This is a sample text with multiple words to demonstrate the word counting feature."]
    ],
    theme=gr.themes.Soft()
)

if __name__ == "__main__":
    demo.launch()
'''

# Save the app file
with open('app.py', 'w') as f:
    f.write(app_code)

print("✅ app.py created successfully!")

✅ app.py created successfully!


In [ ]:
# Create requirements.txt file
requirements = '''gradio>=4.0.0
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("✅ requirements.txt created successfully!")

✅ requirements.txt created successfully!


In [ ]:
# Create README.md file
readme_content = '''---
title: Text Processor
emoji: 📝
colorFrom: blue
colorTo: purple
sdk: gradio
sdk_version: 4.44.0
app_file: app.py
pinned: false
license: mit
---

# Text Processor

A simple Gradio app that:
- Converts text to uppercase
- Counts the number of words

Built with ❤️ using Gradio!
'''

with open('README.md', 'w') as f:
    f.write(readme_content)

print("✅ README.md created successfully!")

✅ README.md created successfully!


## 🔐 Step 5: Set Up Hugging Face Authentication

You'll need a Hugging Face account and token to deploy. Here's how to set it up:

In [ ]:
from huggingface_hub import HfApi, login
import getpass

# Login to Hugging Face
# You can get your token from: https://huggingface.co/settings/tokens
print("🔑 Please enter your Hugging Face token:")
print("Get it from: https://huggingface.co/settings/tokens")

token = getpass.getpass("Enter your HF token: ")
login(token=token)

🔑 Please enter your Hugging Face token:
Get it from: https://huggingface.co/settings/tokens
Enter your HF token: ··········


## 🚀 Step 6: Deploy to Hugging Face Spaces

Now let's deploy our app to Hugging Face Spaces!

In [ ]:
from huggingface_hub import HfApi

def deploy_to_spaces(space_name, username):
    """
    Deploy the current directory to Hugging Face Spaces

    Args:
        space_name: Name for your space (e.g., 'my-text-processor')
        username: Your Hugging Face username
    """
    api = HfApi()

    repo_id = f"{username}/{space_name}"

    try:
        # Create the space
        api.create_repo(
            repo_id=repo_id,
            repo_type="space",
            space_sdk="gradio",
            private=False
        )
        print(f"✅ Space '{repo_id}' created successfully!")

        # Upload files
        api.upload_file(
            path_or_fileobj="app.py",
            path_in_repo="app.py",
            repo_id=repo_id,
            repo_type="space"
        )

        api.upload_file(
            path_or_fileobj="requirements.txt",
            path_in_repo="requirements.txt",
            repo_id=repo_id,
            repo_type="space"
        )

        api.upload_file(
            path_or_fileobj="README.md",
            path_in_repo="README.md",
            repo_id=repo_id,
            repo_type="space"
        )

        print(f"🎉 App deployed successfully!")
        print(f"🌐 Visit your app at: https://huggingface.co/spaces/{repo_id}")

    except Exception as e:
        print(f"❌ Error: {e}")
        print("Make sure you're logged in and the space name doesn't already exist.")

# Example usage (uncomment and modify with your details):
deploy_to_spaces("my-text-processor-demo", "medmekk")

✅ Space 'medmekk/my-text-processor-demo' created successfully!
🎉 App deployed successfully!
🌐 Visit your app at: https://huggingface.co/spaces/medmekk/my-text-processor-demo


## 📋 Step 7: Alternative Deployment Methods

If you prefer, you can also deploy manually through the Hugging Face website:

### Method 1: Web Interface
1. Go to [huggingface.co/new-space](https://huggingface.co/new-space)
2. Choose "Gradio" as the SDK
3. Upload your files (`app.py`, `requirements.txt`, `README.md`)
4. Your app will build and deploy automatically!

### Method 2: Git Clone Method
```bash
# Create the repo and clone it to work locally (best option)
git clone https://huggingface.co/spaces/your-username/your-space-name
cd your-space-name

# Copy your files
cp app.py requirements.txt README.md ./

# Commit and push
git add .
git commit -m "Add Gradio app"
git push
```

## 💡 Best Practices & Tips

### 1. File Structure
Your Hugging Face Space should have:
```
your-space/
├── app.py          # Main application file
├── requirements.txt # Python dependencies
└── README.md       # Space configuration and description
```

### 2. README.md Configuration
The header in README.md is crucial:
- `sdk: gradio` tells HF to use Gradio
- `app_file: app.py` specifies the main file
- `sdk_version` should match your Gradio version

### 3. Requirements.txt Tips
- Pin versions for stability: `gradio==4.44.0`
- Include all dependencies your app needs
- Keep it minimal to reduce build time

### 4. App.py Best Practices
- Always include error handling
- Use descriptive labels and placeholders
- Add examples to showcase functionality
- Test locally before deploying

## 🔧 Troubleshooting Common Issues

### Build Failures
- Check your `requirements.txt` for typos
- Ensure all imports in `app.py` are available
- Check the build logs in your Space's "Logs" tab

### App Not Loading
- Make sure `app.py` has `if __name__ == "__main__": demo.launch()`
- Check that your main interface is named `demo`
- Verify the `app_file` in README.md matches your filename

### Performance Issues
- Optimize your functions for speed
- Consider using Gradio's built-in caching
- For heavy models, consider using GPU spaces

# Local API



You can deploy your model on custom resources such as Google Colab, making it accessible via a public API. This is especially useful when you want to demonstrate your model's capabilities.

Here is an example of how to deploy a text translation model (FLAN-T5) using FastAPI and Ngrok.

You can deploy any other model in the same way.

In [ ]:
!pip install fastapi uvicorn transformers accelerate pyngrok nest-asyncio -qq

This sets up the translation API and loads the pretrained **flan-t5-base** model.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Initialize app
app = FastAPI(title="T5 Translation API", description="Translate English to other languages using FLAN-T5")

# Load model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define request schema
class InputText(BaseModel):
    text: str

# Endpoint for text translation
@app.post("/translate")
def generate_text(data: InputText):
    input_ids = tokenizer(data.text, return_tensors="pt").input_ids.to(device)
    output_ids = model.generate(input_ids)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return {"output": output_text}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Create an Ngrok Account :
- Sign up (or log in) to Ngrok:
👉 https://dashboard.ngrok.com/signup

- Once logged in, go to:
👉 https://dashboard.ngrok.com/get-started/your-authtoken

- Copy your authtoken (it looks like 1pJsd0X....)

- Run this in your Colab notebook (replace <YOUR_AUTHTOKEN>):
```!ngrok config add-authtoken <YOUR_AUTHTOKEN>```


In [ ]:
!ngrok config add-authtoken <YOUR_AUTHTOKEN>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Enable nested event loops for Colab
nest_asyncio.apply()

# Create a public URL
public_url = ngrok.connect(8000)
print(f"🌐 Public URL: {public_url}/docs")

# Run the server
uvicorn.run(app, host="0.0.0.0", port=8000)

🌐 Public URL: NgrokTunnel: "https://ba76-34-150-220-201.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [8522]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     197.230.240.146:0 - "POST /translate HTTP/1.1" 200 OK


In another file in your local computer or another colab notebook, execute the code below to get a response from your endpoint.

In [ ]:
import requests

# Replace this with your actual public ngrok URL
url = "https://ba76-34-150-220-201.ngrok-free.app/translate"

# Input text for the translation model
payload = {
    "text": "translate English to French: What time is the meeting?"
}

# Make the POST request
response = requests.post(url, json=payload)

print("Model Output:", response.json())


# HF inference API

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# Upload entire folder
api.upload_folder(
    folder_path="path/to/your/local/model",
    repo_id="your-username/your-model-name",
    repo_type="model"
)_pretrained("path/to/your/local/model")
tokenizer = AutoTokenizer.from_pretrained("path/to/your/local/model")

# Push to Hub
model.push_to_hub("your-username/your-model-name")
tokenizer.push_to_hub("your-username/your-model-name")

# Challenge

Deploy a simple LLM inference gradio demo